In [47]:
import pandas as pd
import numpy as np


# data loading

I got the description of the columns from this link:
https://www.kaggle.com/datasets/kazanova/sentiment140

In [48]:
train=pd.read_csv("/content/train_data.csv",encoding='latin-1', header=None)
test=pd.read_csv("/content/test_data.csv", encoding='latin-1', header=None)
train.columns=['target','ID','date','flag','user','text']
test.columns=['target','ID','date','flag','user','text']

In [49]:
train.head()

,target,ID,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


# data preprocessing

In [50]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  20000 non-null  int64 
 1   ID      20000 non-null  int64 
 2   date    20000 non-null  object
 3   flag    20000 non-null  object
 4   user    20000 non-null  object
 5   text    20000 non-null  object
dtypes: int64(2), object(4)
memory usage: 937.6+ KB


Yay! We don't have any missing values

In [51]:
train["target"].unique()

array([0, 4])

In [52]:
train['target'].replace({4:1},inplace=True)

In [53]:
train["target"].unique()

array([0, 1])

In [54]:
#print some random sentences to understand how to preprocess the text
random=np.random.randint(0, 20000, size=20)
for i in random:
  print(train.loc[i,'text'])
  print('----------------------------------')

is as work! 
----------------------------------
@cali_chica Ooh that makes 2 of us that's happy. Going 2 have an Orchata later this afternoon I&quot;l have 1 4 u if u have a quesidilla 4 me 
----------------------------------
@DanielleHerb Yay, cool I'm looking forward to getting to know you too. And let's start a #stopautodms trend 
----------------------------------
@McFlyStarGirl21 Ahh good good! Well i get them too so i know how ya feel!  x
----------------------------------
Time Off Day 2: Just got back from my time out, i am so tired of being skint, no good news at the bank  May watch another zombie film now.
----------------------------------
. . . . I forgot the password for twitter . . . Spent last two days trying to remember it. .at last gave up  . . . Reset 
----------------------------------
He's the reason for the teardrops on my guitar the only one who has enough of me to break my heart 
----------------------------------
@nsha twitter easier, send once all receive 
-----

In [55]:
import re

def preprocessor(text):
  text=re.sub(r'\s*@\w+', '', text)#remove words starting with @ from a text
  text=text.lower()
  return text


train['text']=train['text'].apply(preprocessor)

In [56]:
#test the preprocessor function
for i in random:
  print(train.loc[i,'text'])
  print('----------------------------------')

is as work! 
----------------------------------
 ooh that makes 2 of us that's happy. going 2 have an orchata later this afternoon i&quot;l have 1 4 u if u have a quesidilla 4 me 
----------------------------------
 yay, cool i'm looking forward to getting to know you too. and let's start a #stopautodms trend 
----------------------------------
 ahh good good! well i get them too so i know how ya feel!  x
----------------------------------
time off day 2: just got back from my time out, i am so tired of being skint, no good news at the bank  may watch another zombie film now.
----------------------------------
. . . . i forgot the password for twitter . . . spent last two days trying to remember it. .at last gave up  . . . reset 
----------------------------------
he's the reason for the teardrops on my guitar the only one who has enough of me to break my heart 
----------------------------------
 twitter easier, send once all receive 
----------------------------------
 ahh :s i dnt k

In [57]:
#tokenizer

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Initialize the stemmer
stemmer = PorterStemmer()

# Define a custom tokenizer that includes stemming
def stemmed_words(doc):
    return [stemmer.stem(word) for word in word_tokenize(doc)]


In [58]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
#tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



tfidf=TfidfVectorizer(use_idf=True,
                       norm='l2',
                       smooth_idf=True,
                       stop_words=stop,
                       tokenizer=stemmed_words,
                      max_features=1000)



In [74]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

nb_tfidf = Pipeline([('tfidf', tfidf),
                     ('mnb', MultinomialNB())])

In [76]:
nb_tfidf.fit(train['text'],train['target'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...],
                                 tokenizer=<function stemmed_words at 0x7d0e65445c60>)),
                ('mnb', MultinomialNB())])

In [77]:
from sklearn import metrics
predictions=nb_tfidf.predict(test['text'])
print(metrics.classification_report(test['target'], predictions))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68       177
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00       139
           4       0.00      0.00      0.00       182

    accuracy                           0.24       498
   macro avg       0.17      0.17      0.17       498
weighted avg       0.24      0.24      0.24       498



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

##linear regression

In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

lr_tfidf = Pipeline([('tfidf', tfidf),
                     ('mnb', LogisticRegression())])
lr_tfidf.fit(train['text'],train['target'])
from sklearn import metrics
predictions=lr_tfidf.predict(test['text'])
print(metrics.classification_report(test['target'], predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.67      0.71      0.69       177
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00       139
           4       0.00      0.00      0.00       182

    accuracy                           0.25       498
   macro avg       0.17      0.18      0.17       498
weighted avg       0.24      0.25      0.25       498



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_